# Interaction with UI.py using JupyterNotebook #

This project is the second assignment for Research Track 1. It is a simulation of a motion planning of a mobile robot in an environment. Additionally, with the use of jupyternotebook the user can control the motion of the robot. In addition, a visualization of the robot movement along with the target position will be plotted. For the user to view the distance of the obstacle from the robot, he/she can click on the button "display distance". A bar plot is also available to compare the number of goals achieved vs the number of goals cancelled. For further details scroll down, please. 

It is important to note that the user can interact with the robot using jupyter notebook while also being able to set target positions for the robot to reach similarly in Research Track assignment 2.



Import the necessary libraries and declaration of variables that will be used later in the code.

In [ ]:
import rospy
import actionlib
import assignment_2_2022.msg
import time
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
from assignment_2_2022.msg import RobotPose
from sensor_msgs.msg import LaserScan
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from ipywidgets import interact, interactive, fixed, interact_manual
import jupyros as jr
from std_srvs.srv import Trigger
from std_msgs.msg import String
from jupyros import ros3d
from jupyros import ros3d 
import os
%use of FuncAnimation for plotting the robot position
import tf
from tf.transformations import quaternion_matrix
from matplotlib.animation import FuncAnimation
from IPython.display import display


linear_x = 0
linear_y = 0
#distance of the first obstacle from the robt
min_obstacle_dist = 0


Mostly, this section was borrowed from the second assignment of Research Track 1. The function laser_callback was added to update the distance of the obstacle from the robot. 

In [ ]:
def odom_callback(data):
    global linear_x, linear_y
    linear_x = data.twist.linear.x
    linear_y = data.twist.linear.y

#function o update the distance of the obstacle from the robot
def laser_callback(data):
    global min_obstacle_dist
    min_obstacle_dist = min(data.ranges)
    

class UpdatePositionsClient():
    def __init__(self):
       # Initializes _client node
       self._action_client = actionlib.SimpleActionClient('reaching_goal', assignment_2_2022.msg.PositionsAction)
       self._publisher = rospy.Publisher('robot_pose', RobotPose, queue_size=10)

    # Waits for server to be available, then sends goal
    def send_goal(self, x, y):
        goal_msg = assignment_2_2022.msg.PositionsGoal()
        goal_msg.target_x = x
        goal_msg.target_y = y
        rospy.loginfo('Waiting for server...')

        self._action_client.wait_for_server()

        # Returns future to goal handle; client runs feedback_callback after sending the goal
        self._send_goal_future = self._action_client.send_goal(goal_msg, active_cb=self.goal_response_callback, feedback_cb=self.feedback_callback, done_cb=self.get_result_callback)

        rospy.loginfo("Goal sent!")

    # Run when client accepts goal
    def goal_response_callback(self):
        rospy.loginfo('Goal accepted :)')

    # Run when action server sends feedback
    def feedback_callback(self, feedback_msg):
        rospy.loginfo('Received feedback x at: {0}'.format(feedback_msg.current_x))
        rospy.loginfo('Received feedback y at: {0}'.format(feedback_msg.current_y))

        # Create custom message
        msg = RobotPose()
        # Populate message fields with current position and velocity data
        msg.current_x = feedback_msg.current_x
        msg.current_y = feedback_msg.current_y 
        msg.current_vx = linear_x
        msg.current_vy = linear_y
        # Publish message on topic
        self._publisher.publish(msg)
        rospy.loginfo('Published robot position and velocity: x={0}, y={1}, vx={2}, vy={3}'.format(feedback_msg.current_x, feedback_msg.current_y, linear_x, linear_y))

    # Run when client sends final result
    def get_result_callback(self, state, result):
        # Show log and exit node
        if result.is_finished:
            rospy.loginfo('Result: {0}'.format(result.is_finished))
        else:
            rospy.loginfo("Goal failed")

    # Method to allow the user to cancel target positions x and y that were sent to the server
    def cancel_goal(self):
        self._action_client.cancel_goal())
    
    # Method to return the distance of ther obstacle from the robot
    def get_min_obstacle_dist(self):
        return min_obstacle_dist


Below are event handlers for the buttons for when they are clicked. There are 4 buttons: forward, backward, left and right. When of these buttons is clicked a new position target is sent to the robot

In [ ]:
# these are event handlers for the buttons for when they are clicked
def forward_clicked(b):
    action_client.send_goal(0.5, 0)

def backward_clicked(b):
    action_client.send_goal(-0.5, 0)

def left_clicked(b):
    action_client.send_goal(0, 0.5)

def right_clicked(b):
    action_client.send_goal(0, -0.5)


Mostly, the section below was borrowed from the second assignment of Research Track 1 to ask from the user the target/goal position. The user can also cancel the goal. Additionally, there is the creation of the 4 buttons for the user to control the motion of the robot. 

In [ ]:
def main(args=None):

    # in this section below we are building hte buttons with their respective description
    forward_button = widgets.Button(description='Forward')
    backward_button = widgets.Button(description='Backward')
    left_button = widgets.Button(description='Left')
    right_button = widgets.Button(description='Right')

    # Event handlers for the buttons when they are clicked by user (refer to previous cell)
    forward_button.on_click(forward_clicked)
    backward_button.on_click(backward_clicked)
    left_button.on_click(left_clicked)
    right_button.on_click(right_clicked)

    # displaying the buttons created
    display(forward_button, backward_button, left_button, right_button)
    
    # Subscribe to the /odom topic using a subscriber
    odom_sub = rospy.Subscriber('/odom', Odometry, odom_callback)

    action_client = UpdatePositionsClient()


    
    while not rospy.is_shutdown():

        # taking the target positions from user 
        try:
            x, y = float(input("Enter target positions x and y, seperated by a white space: ").split())
            print("Target x is {} and Target y is {}".format(x,y))
            print()
        except ValueError:#in case the user enters wrong values 
            print("Invalid input. Please enter two numeric values.")
            continue
        action_client = UpdatePositionsClient()
        # Sends goal and waits until it's completed
        action_client.send_goal(x,y) 


        # Check if user wants to cancel the goal
        cancel_input = input("if you want to cancel then type 'cancel', or type any other key if you want to continue: ")
        if cancel_input == "cancel": #user typed cancel 
                action_client.cancel_goal()
                print("Your Goal got cancelled!")
                continue #essential!! to allow the program to go back to the beginning of the loop and ask for new targets
        
        rospy.spin()

if __name__ == '__main__':
# Init ROS1 and give the node a name
   rospy.init_node("UI")
   main()  




CLASS visualiser to plot the robot changing position versus the target position set by the user. 

In [ ]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.robot_ln, = plt.plot([], [], 'bo', label='Robot Position')
        self.target_ln, = plt.plot([], [], 'ro', label='Target Positions')
        self.x_data_robot, self.y_data_robot = [], []
        self.x_data_target, self.y_data_target = [], []
        self.ax.legend()

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.robot_ln, self.target_ln

    def odom_callback(self, msg):
        self.y_data_robot.append(msg.pose.pose.position.y)
        self.x_data_robot.append(msg.pose.pose.position.x)

    def update_plot(self, frame):
        self.robot_ln.set_data(self.x_data_robot, self.y_data_robot)
        self.target_ln.set_data(self.x_data_target, self.y_data_target)
        return self.robot_ln, self.target_ln




In [ ]:
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

A delete button is added to give the option to the user to clear the previous plot. So when clicked, it will close the plot window

In [ ]:
def delete_plot_clicked(b):
    plt.close()

delete_plot = widgets.Button(description='Delete Plot')
delete_plot.on_click(delete_plot_clicked)
display(delete_plot)


The cell below create a button and textbox to give the option to user to to display the distance of the obstacle from the user 

In [ ]:
# this is a text box to display to the user the distance of the obstacle from the robot
distance_box = widgets.Text(value='Distance: 0', disabled=True)

# if the user want to check the distance, he/she has to click on the button "display distance"
display_button = widgets.Button(description='Display Distance')

# Callback function for laserscan 
def laserscan_callback(msg):
    distance = msg.ranges[0] 
    distance_box.value = f'Distance: {distance}'

#subsribe to topic /scan
sub = rospy.Subscriber('/scan', LaserScan, laserscan_callback)

# Callback for when button is clicked 
def display_button_clicked(button):
    
    pass


display_button.on_click(display_button_clicked)

display(widgets.VBox([distance_textbox, display_button]))


Bar plot to display success/fail goals

In [ ]:
def display_goals(button):
    rospy.wait_for_service('/check_status')
    try:
        check_status_service = rospy.ServiceProxy('/check_status', Trigger)
        response = check_status_service()
        counter_cancelled_srv = response.counter_cancelled_srv
        counter_success_srv = response.counter_success_srv

        # bar plot
        labels = ['cancelled goals', 'success goals']
        values = [counter_cancelled_srv, counter_success_srv]
        x_pos = np.arange(len(labels))

        plt.bar(x_pos, values, align='center', alpha=0.5)
        plt.xticks(x_pos, labels)
        plt.ylabel('# of Goals')
        plt.title('success vs fail goals')

        plt.show()
    except rospy.ServiceException as e:
        rospy.logerr('Service call failed: {0}'.format(e))



display_button = widgets.Button(description='Display Success/Fail Goals')
display_button.on_click(display_goals)
display(display_button)


ROS3D communicates with ROS via websocket. This communication is configured through the jupyter widgets protocol, but you are also required to run the “rosbridge websocket” package in your ROS environment (or launch file).
IMPORTANT, you need to make sure that you have ros-noetic-rosbridge-suite and ros-noetic-tf2-webrepublisher:
Run
apt-get install ros-noetic-rosbridge-suite
apt-get install ros-noetic-tf2-web-republisher

Use ROS3d to get visualizations to the viewer. ROS3D contains classes for a LaserScan, a Robot (URDFModel), occupancy grid (OccupancyGrid), regular and interactive markers (Marker and InteractiveMarker), Pose, Polygon, PoseArray, Path, and PointCloud

In [ ]:
v = ros3d.Viewer()
rc = ros3d.ROSConnection(url="ws://localhost:9090")
tf_client = ros3d.TFClient(ros=rc, fixed_frame='odom')
laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client,
g = ros3d.GridModel()
v.objects = [g, laser_view, urdf]
v


This will now attempt to connect to the websocket at ws://localhost:9090 . And if successful, it should receive data from /scan topic and display it in the 3D viewer.